In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
import plotly.express as px
from wordcloud import WordCloud
import plotly.graph_objects as go
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import plotly.express as px
import string
import warnings
warnings.filterwarnings('ignore')

In [16]:
df1 = pd.read_csv('/content/drive/MyDrive/ds-ml/text-classification-bert-and-tf/train.csv',index_col='id')

In [17]:
test = pd.read_csv('/content/drive/MyDrive/ds-ml/text-classification-bert-and-tf/test.csv',index_col='id')

In [18]:
test

,keyword,location,text
id,,,
0,NaN,NaN,Just happened a terrible car crash
2,NaN,NaN,"Heard about #earthquake is different cities, s..."
3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...
10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...


In [43]:
from sklearn.base import BaseEstimator, TransformerMixin
import string
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
import regex as re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

class PreprocessText(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.punc = string.punctuation
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_copy = X.copy()
        X_copy['text'] = X_copy['text'].apply(self.preprocess_text)
        return X_copy

    def preprocess_text(self, text):
        # Lowercase the text
        text = text.lower()

        # Remove punctuation
        text = self.remove_punc(text)

        # Remove stopwords
        text = self.remove_stopwords(text)

        # Lemmatization
        text = self.lemmatize_text(text)

        return text

    def remove_punc(self, text):
        return text.translate(str.maketrans('', '', self.punc))

    def remove_stopwords(self, text):
        words = word_tokenize(text)
        filtered_words = [word for word in words if word not in self.stop_words]
        return ' '.join(filtered_words)

    def lemmatize_text(self, text):
        words = word_tokenize(text)
        lemmatized_words = [self.lemmatizer.lemmatize(word) for word in words]
        return ' '.join(lemmatized_words)

# Usage
preprocessor = PreprocessText()
df1['clean_text'] = preprocessor.fit_transform(df1[['text']])
test['clean_text'] = preprocessor.fit_transform(test[['text']])


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier  # New import

# Splitting data into train and test sets
train_df, test_df = train_test_split(df1, test_size=0.2, random_state=42)

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'Multinomial Naive Bayes': MultinomialNB(),
    'Random Forest': RandomForestClassifier(),
    'Support Vector': SVC(),
    'Decision Tree': DecisionTreeClassifier(),
    'XGBoost': XGBClassifier(),
    'LightGBM': LGBMClassifier(),
    'Gradient Boosting': GradientBoostingClassifier()
}

# Iterate over models
for model_name, model in models.items():
    print(f"Processing model: {model_name}")

    # Define pipeline
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', model)
    ])

    # Train pipeline
    pipeline.fit(train_df['clean_text'], train_df['target'])

    # Make predictions
    predictions = pipeline.predict(test_df['clean_text'])

    # Calculate accuracy
    accuracy = accuracy_score(test_df['target'], predictions)
    print(f"Accuracy for {model_name}: {accuracy}")

    # Save predictions to CSV
    predictions_df = pd.DataFrame({'target': predictions}, index=test_df.index)
    predictions_df.to_csv(f"/content/drive/MyDrive/ds-ml/text-classification-bert-and-tf/predictions/submit_{model_name.lower().replace(' ', '_')}_v2.csv")


Processing model: Logistic Regression
Accuracy for Logistic Regression: 0.8003939592908733
Processing model: Multinomial Naive Bayes
Accuracy for Multinomial Naive Bayes: 0.7951411687458962
Processing model: Random Forest
Accuracy for Random Forest: 0.778069599474721
Processing model: Support Vector
Accuracy for Support Vector: 0.7984241628365069
Processing model: Decision Tree
Accuracy for Decision Tree: 0.7235718975705844
Processing model: XGBoost
Accuracy for XGBoost: 0.7741300065659882
Processing model: LightGBM
[LightGBM] [Info] Number of positive: 2622, number of negative: 3468
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020171 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8319
[LightGBM] [Info] Number of data points in the train set: 6090, number of used features: 610
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.430542

In [21]:
models = {
    'Logistic Regression': (LogisticRegression(), {'clf__C': [0.1, 1, 10]}),
    'Multinomial Naive Bayes': (MultinomialNB(), {'clf__alpha': [0.1, 1, 10]}),
    'Random Forest': (RandomForestClassifier(), {'clf__n_estimators': [50, 100, 200]}),
    'Support Vector': (SVC(), {'clf__C': [0.1, 1, 10], 'clf__gamma': ['scale', 'auto']}),
    'Decision Tree': (DecisionTreeClassifier(), {'clf__max_depth': [None, 10, 50, 100]}),
    'XGBoost': (XGBClassifier(), {'clf__n_estimators': [50, 100, 200]}),
    'LightGBM': (LGBMClassifier(), {'clf__n_estimators': [50, 100, 200]})
}

In [49]:
test.columns

Index(['keyword', 'location', 'text', 'clean_text'], dtype='object')

In [48]:
test.columns

Index(['keyword', 'location', 'text', 'clean_text'], dtype='object')

In [53]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Iterate over models and perform hyperparameter tuning
for model_name, (model, param_grid) in models.items():
    print(f"Processing model: {model_name}")
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', model)
    ])

    # Perform GridSearchCV for hyperparameter tuning
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(train_df['clean_text'], train_df['target'])

    # Print the best hyperparameters
    print(f"Best hyperparameters for {model_name}: {grid_search.best_params_}")

    # Predict on the test set
    predictions = grid_search.predict(test['clean_text'])
    predictions_df = pd.DataFrame(predictions, columns=['target'])

    # accuracy = accuracy_score(test_df['target'], predictions)
    # print(f"Accuracy for {model_name}: {accuracy:.2f}")
    predictions_df = pd.DataFrame({'target': predictions}, index=test.index)
    # Save predictions to CSV file
    predictions_df.to_csv(f"/content/drive/MyDrive/ds-ml/text-classification-bert-and-tf/predictions/submit_{model_name.lower().replace(' ', '_')}_v3.csv")

Processing model: Logistic Regression
Best hyperparameters for Logistic Regression: {'clf__C': 1}
Processing model: Multinomial Naive Bayes
Best hyperparameters for Multinomial Naive Bayes: {'clf__alpha': 1}
Processing model: Random Forest
Best hyperparameters for Random Forest: {'clf__n_estimators': 200}
Processing model: Support Vector
Best hyperparameters for Support Vector: {'clf__C': 1, 'clf__gamma': 'scale'}
Processing model: Decision Tree
Best hyperparameters for Decision Tree: {'clf__max_depth': 50}
Processing model: XGBoost
Best hyperparameters for XGBoost: {'clf__n_estimators': 200}
Processing model: LightGBM
[LightGBM] [Info] Number of positive: 2097, number of negative: 2775
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018171 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5899
[LightGBM] [Info] Number of data points in t